In [93]:
# importing the required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from IPython.display import display
import re

warnings.filterwarnings('ignore')
pd.set_option("display.max_rows",500)
pd.set_option("display.max_columns",500)
pd.set_option('display.max_colwidth', None)

In [94]:
# loading the data
df = pd.read_csv("data/customer_data_RFM_features.csv",parse_dates=['DateTime'])

In [95]:
# printing the first 5 rows
df.head(5)

,User_id,Session_id,DateTime,Category,SubCategory,Action,Quantity,Rate,Total Price,Recency,Frequency,Monetary,R,F,M,Group,Score,Loyalty
0,52243841613,d76fde-8bb3-4e00-8c23,2019-01-10 10:20:00,Electronic Appliances,Speakers,first_app_open,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52243841613,33dfbd-b87a-4708-9857,2019-01-10 10:22:00,Electronic Appliances,Speakers,search,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,57314161118,6511c2-e2e3-422b-b695,2019-01-10 14:00:00,Men's Fashion,Jeans,search,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,57314161118,90fc70-0e80-4590-96f3,2019-01-10 14:07:00,Men's Fashion,Jeans,product_view,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,57314161118,bd7419-2748-4c56-95b4,2019-01-10 14:12:00,Men's Fashion,Jeans,read_reviews,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
# checking the data shape
print(f"The shape of the dataframe is {df.shape}")

The shape of the dataframe is (2090, 18)


In [97]:
# basic check about the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2090 entries, 0 to 2089
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   User_id      2090 non-null   int64         
 1   Session_id   2090 non-null   object        
 2   DateTime     2090 non-null   datetime64[ns]
 3   Category     2090 non-null   object        
 4   SubCategory  2090 non-null   object        
 5   Action       2090 non-null   object        
 6   Quantity     2090 non-null   float64       
 7   Rate         2090 non-null   float64       
 8   Total Price  2090 non-null   float64       
 9   Recency      1303 non-null   float64       
 10  Frequency    1303 non-null   float64       
 11  Monetary     1303 non-null   float64       
 12  R            1303 non-null   float64       
 13  F            1303 non-null   float64       
 14  M            1303 non-null   float64       
 15  Group        1303 non-null   float64       
 16  Score 

## Feature Engineering

#### Date Level Features

In [98]:
# creating date level features like date, day_of_week, day_of_month
df['Date'] = df['DateTime'].dt.date
df['DayOfWeek'] = df['DateTime'].dt.dayofweek
df['DayOfMonth'] = df['DateTime'].dt.day

#### User activity features

##### 1.No of Days Active 

In [99]:
# creating a feature that says how many days the user were active on our platform
days_active = df.groupby('User_id').agg({'Date':'nunique'}).reset_index()
days_active.rename(columns={"Date":"days_active"},inplace=True)

days_active.head()

,User_id,days_active
0,51443820326,1
1,51448162784,3
2,51455512201,3
3,51462598385,5
4,51463622150,3


##### 2. Avg time between purchases

In [100]:
# function to remove NaN from the list
def compute_avg_purchase_gap(lst):
    assert type(lst)==list
    if len(lst)==1:
        return np.nan
    # removing the NaN values
    lst_new = list(filter(lambda x: not pd.isna(x), lst))
    # returning the avg of the list (since we want avg purchase gap)
    return sum(lst_new)/len(lst_new)


# definin a function which gets avg time between purchases
def get_avg_time_between_purchases(data,action="Action",user_id="User_id",date_time="DateTime"):
    """ 
    data      : dataframe
    action    : name of the action column in the dataframe
    user_id   : name of the user_id column in the dataframe
    date_time : datetime of the action
    """
   # creating a copy of the dataframe
    dff = data.copy()
    # filtering the rows with only purchases
    purchases = dff[dff[action]=="purchase"]
    # selecting only the user id and datetime column
    purchases = purchases[[user_id,date_time]]
    # sorting the purchases by user_id and dattime
    purchases.sort_values(by=[user_id,date_time],ascending=[True,True],inplace=True)
    # creating a purchases_duration column
    purchases.head(2)
    # creating a purchases_duration column
    purchases['Purchase Gap'] = purchases.groupby([user_id])[date_time].diff().dt.total_seconds()
    # converting the timegap to days
    purchases['Purchase Gap'] = np.round(purchases['Purchase Gap']/(3600*24),0)
    # grouping the multiple purchase gaps for an user into a single list (if a cusomter does only single purchase then purchase gap would be NaN, if a person does two purchase then the list contains [nan,purchasegap])
    purchases = pd.DataFrame(purchases.groupby(user_id)["Purchase Gap"].apply(list)).reset_index()
    # applying the compute_avg_purchase_gap func
    purchases["Avg Purchase Gap"] = purchases["Purchase Gap"].apply(compute_avg_purchase_gap)
    
    return purchases

In [101]:
# computing the avg time between purchases for each user
avg_purchase_time = get_avg_time_between_purchases(df,"Action","User_id","DateTime")
avg_purchase_time.head()

,User_id,Purchase Gap,Avg Purchase Gap
0,51448162784,"[nan, 184.0]",184.0
1,51455512201,[nan],NaN
2,51462598385,"[nan, 56.0]",56.0
3,51463622150,"[nan, 29.0]",29.0
4,51465529647,"[nan, 7.0]",7.0


##### 3. Cart to purchase ratio

In [102]:
def get_ratios(data,level,action_col,action1,action2):
    """
    data       : name of the dataframe
    level      : the level in which we want to calculate ratios (if we want it in userlevel pass 'user_id column') (if we want it in category level pass category )
    action_col : name of the col that contains the actions
    action1    : action of interest 1
    action2    : action of interest 2 purchase

    this fuctino returns ratios of action1:action2
    """
    dff = data.copy()
    # creating a dataframe which has no of action1 done by each users
    col1 = dff.groupby(level)[action_col].value_counts().loc[:, action1].reset_index()
    # creating a dataframe which has no of action1 done by each users
    col2 = dff.groupby(level)[action_col].value_counts().loc[:, action2].reset_index()
    # combining both dfs
    ratios = pd.merge(col1, col2, on=level, how="left", suffixes=("_"+action1,"_"+action2) )
    # creating the ratio column which contains the action2/action1 value                 
    ratios[f'{action1}_to_{action2}_ratios']=ratios[f"count_{action2}"]/ratios[f"count_{action1}"]
    # filling NaNs with 0
    ratios.fillna(0,inplace=True)
    
    return ratios
    

In [103]:
# creating cart_purchase ratio for each user
cart_to_purchase = get_ratios(df,"User_id",'Action',"add_to_cart","purchase")
cart_to_purchase.head()

,User_id,count_add_to_cart,count_purchase,add_to_cart_to_purchase_ratios
0,51448162784,2,2.0,1.000000
1,51455512201,1,1.0,1.000000
2,51462598385,2,2.0,1.000000
3,51463622150,2,2.0,1.000000
4,51465529647,3,2.0,0.666667


##### 4. Wishlist to purchase ratio

In [104]:
# creating wish_list_to_purchase ratio for each user
wishlist_to_purchase = get_ratios(df,"User_id",'Action',"add_to_wishlist","purchase")
wishlist_to_purchase.head()

,User_id,count_add_to_wishlist,count_purchase,add_to_wishlist_to_purchase_ratios
0,51463622150,1,2.0,2.00
1,51477448981,2,1.0,0.50
2,51517997197,2,2.0,1.00
3,51526464449,1,2.0,2.00
4,51565158157,4,3.0,0.75


In [105]:
# creating wish_list_click_to_purchase ratio for each user
wishlist_click_to_purchase = get_ratios(df,"User_id",'Action',"click_wishlist_page","purchase")
wishlist_click_to_purchase.head()

,User_id,count_click_wishlist_page,count_purchase,click_wishlist_page_to_purchase_ratios
0,51448162784,1,2.0,2.0
1,51463622150,1,2.0,2.0
2,51465529647,1,2.0,2.0
3,51468519074,1,1.0,1.0
4,51475146821,1,0.0,0.0


In [106]:
df.Action.unique()

array(['first_app_open', 'search', 'product_view', 'read_reviews',
       'add_review', 'add_to_cart', 'checkout', 'purchase',
       'add_to_wishlist', 'click_wishlist_page', 'product_review'],
      dtype=object)

##### 5. Paths

In [107]:
# defining a function to get the latest path of each user
def get_paths(data,date_time,user_id,action_col):
    """
    data       : name of the dataframe
    user_id    : name of the col that contains the userid
    action_col : name of the col that contains the actions
    
    this fuctino returns the latest path of each user
    """
    
    # creating a temp copy 
    temp = data.copy()
    
    # sorting the temp df by datetime column in ascending order
    temp.sort_values(by=date_time,ascending=True)
    
    # filtering the rows with add_to_carts action
    add_to_carts = temp[temp[action_col]=="add_to_cart"] 
    
    # finding the last add_to_cart event for each user
    latest_add_to_carts = add_to_carts.groupby(user_id)[date_time].max()
    
    # filtering the rows with actions that was done before the last add to cart action of each user
    temp = temp[temp.DateTime<=temp[user_id].map(latest_add_to_carts)] # now this dataframe consists of actions before the last addtocart event for each users
    
    # picking the last three actions done before add_to_cart by each users
    paths = temp.groupby(user_id)[action_col].apply(list).reset_index(name='Actions_list')
    
    # Joining all the elements of the lists
    paths['Actions_list'] = paths['Actions_list'].apply(lambda x: ' '.join(x))
    
    # applying regex to find the last 3 actions 
    last_3_actions = r'((?:\S+\s+){0,3}\badd_to_cart)'
    paths["Last_3_Actions"] = paths['Actions_list'].apply(lambda x: re.findall(last_3_actions,x))
    
    # getting the latest path of the user
    paths["user_path"] = paths["Last_3_Actions"].apply(lambda x:x[-1])
    paths["user_path"] = paths["user_path"].apply(lambda x:"-->".join(x.split()))
    
    # dropping the unwanted columns
    paths = paths[[user_id,"user_path"]]

    return paths
        
    

In [108]:
# getting the paths for each users
paths = get_paths(df,"DateTime","User_id","Action")
paths.head()

,User_id,user_path
0,51448162784,search-->read_reviews-->add_to_cart
1,51455512201,search-->product_view-->read_reviews-->add_to_cart
2,51462598385,search-->product_view-->read_reviews-->add_to_cart
3,51463622150,purchase-->search-->product_view-->add_to_cart
4,51465529647,search-->add_to_cart


In [109]:
print(f"There are {paths['user_path'].nunique()} no unique path in which people are adding the product to the cart\n")

There are 44 no unique path in which people are adding the product to the cart



In [110]:
paths['user_path'].value_counts(ascending=True).head()

user_path
add_to_wishlist-->search-->product_view-->add_to_cart           1
search-->click_wishlist_page-->add_to_cart                      1
add_to_wishlist-->click_wishlist_page-->search-->add_to_cart    1
product_view-->search-->product_view-->add_to_cart              1
checkout-->search-->product_view-->add_to_cart                  1
Name: count, dtype: int64

as you can see lot of paths are taken only very few times and also we have 44 no of unique paths so lets combine the minority paths into a other group

In [111]:
def combine_paths(data,path_col):
    # getting the top 10 paths 
    top_10_paths = data['user_path'].value_counts(ascending=False).head(10).index.to_list()
    # combining the minority paths
    data['user_path'] = data['user_path'].apply(lambda x:x if x in top_10_paths else "others"  )
    return data 

paths = combine_paths(paths,"user_path")

In [112]:
# counting the paths
paths["user_path"].value_counts()

user_path
others                                                             67
product_view-->read_reviews-->click_wishlist_page-->add_to_cart    29
read_reviews-->click_wishlist_page-->add_to_cart                   28
search-->product_view-->read_reviews-->add_to_cart                 25
click_wishlist_page-->add_to_cart                                  24
add_to_cart                                                        24
read_reviews-->add_to_cart                                         11
product_view-->read_reviews-->add_to_cart                           9
search-->read_reviews-->click_wishlist_page-->add_to_cart           9
read_reviews-->product_view-->click_wishlist_page-->add_to_cart     8
search-->product_view-->click_wishlist_page-->add_to_cart           6
Name: count, dtype: int64

#### Category/SubCategory level features

##### 1. Cart to Purchase (Category)

In [113]:
# calculating the cart-to-purchase ratio for each categories
cart_to_purchase_ratio_category =  get_ratios(df,"Category","Action","add_to_cart","purchase").rename(columns={"add_to_cart_to_purchase_ratios":"cart_to_purchase_ratios_category"})
cart_to_purchase_ratio_category.head()

,Category,count_add_to_cart,count_purchase,cart_to_purchase_ratios_category
0,Accessories,15,12.0,0.8
1,Air Conditioner,1,2.0,2.0
2,Appliances,1,1.0,1.0
3,Artwork,1,0.0,0.0
4,Audio,1,0.0,0.0


##### 2. Cart to Purchase (SubCategory)

In [114]:
# calculating the cart-to-purchase ratio for each categories
cart_to_purchase_ratio_subcategory =  get_ratios(df,"SubCategory","Action","add_to_cart","purchase").rename(columns={"add_to_cart_to_purchase_ratios":"cart_to_purchase_ratios_subcategory"})
cart_to_purchase_ratio_subcategory.head()

,SubCategory,count_add_to_cart,count_purchase,cart_to_purchase_ratios_subcategory
0,Air Tight Container,1,0.0,0.0
1,Air coolers,1,1.0,1.0
2,Amazon Alexa,1,0.0,0.0
3,Amplifier,1,1.0,1.0
4,Audio Speakers,1,0.0,0.0


##### 3. Wishlist to Purchase (Category)

In [115]:
# calculating the cart-to-purchase ratio for each categories
wishlist_to_purchase_ratio_category =  get_ratios(df,"Category","Action","add_to_wishlist","purchase").rename(columns={"add_to_wishlist_to_purchase_ratios":"wishlist_to_purchase_ratios_category"})
wishlist_to_purchase_ratio_category.head()

,Category,count_add_to_wishlist,count_purchase,wishlist_to_purchase_ratios_category
0,Accessories,3,12,4.0
1,Baby Care,1,1,1.0
2,Bed room,1,5,5.0
3,Digital Appliances,1,2,2.0
4,Digital Devices,5,7,1.4


##### 4. Wishlist to Purchase (SubCategory)

In [116]:
# calculating the cart-to-purchase ratio for each categories
wishlist_to_purchase_ratio_subcategory =  get_ratios(df,"SubCategory","Action","add_to_wishlist","purchase").rename(columns={"add_to_wishlist_to_purchase_ratios":"wishlist_to_purchase_ratios_subcategory"})
wishlist_to_purchase_ratio_subcategory.head()

,SubCategory,count_add_to_wishlist,count_purchase,wishlist_to_purchase_ratios_subcategory
0,Books,2,4,2.000000
1,Bp monitors,1,1,1.000000
2,Bumpers,1,1,1.000000
3,Cases,3,2,0.666667
4,Charging wire,1,3,3.000000


##### 5. Click Wishlist to Purchase (Category)

In [117]:
# calculating the cart-to-purchase ratio for each categories
click_wishlist_to_purchase_ratio_category =  get_ratios(df,"Category","Action","click_wishlist_page","purchase").rename(columns={"click_wishlist_page_to_purchase_ratios":"click_wishlist_to_purchase_ratios_category"})
wishlist_to_purchase_ratio_subcategory.head()
click_wishlist_to_purchase_ratio_category.head()

,Category,count_click_wishlist_page,count_purchase,click_wishlist_to_purchase_ratios_category
0,Accessories,14,12.0,0.857143
1,Appliances,3,1.0,0.333333
2,Artwork,1,0.0,0.000000
3,Audio,1,0.0,0.000000
4,Baby Care,4,1.0,0.250000


##### 6. Click Wishlist to Purchase (SubCategory)

In [118]:
# calculating the cart-to-purchase ratio for each subcategories
click_wishlist_to_purchase_ratio_subcategory =  get_ratios(df,"SubCategory","Action","click_wishlist_page","purchase").rename(columns={"click_wishlist_page_to_purchase_ratios":"click_wishlist_to_purchase_ratios_subcategory"})
click_wishlist_to_purchase_ratio_subcategory.head()

,SubCategory,count_click_wishlist_page,count_purchase,click_wishlist_to_purchase_ratios_subcategory
0,Air Tight Container,1,0.0,0.0
1,Air coolers,1,1.0,1.0
2,Amazon Alexa,1,0.0,0.0
3,Audio Speakers,1,0.0,0.0
4,Baby Shampoo,1,0.0,0.0


##### 7. View to Purchase (Category)

In [119]:
# calculating the cart-to-purchase ratio for each subcategories
view_to_purchase_ratio_category =  get_ratios(df,"Category","Action","product_view","purchase").rename(columns={"product_view_to_purchase_ratios":"product_view_to_purchase_ratios_category"})
view_to_purchase_ratio_category.head()

,Category,count_product_view,count_purchase,product_view_to_purchase_ratios_category
0,Accessories,9,12.0,1.333333
1,Air Conditioner,7,2.0,0.285714
2,Artwork,1,0.0,0.000000
3,Audio,1,0.0,0.000000
4,Baby Care,3,1.0,0.333333


##### 8. View to Purchase (SubCategory)

In [120]:
# calculating the cart-to-purchase ratio for each subcategories
view_to_purchase_ratio_subcategory =  get_ratios(df,"SubCategory","Action","product_view","purchase").rename(columns={"product_view_to_purchase_ratios":"product_view_to_purchase_ratios_subcategory"})
view_to_purchase_ratio_subcategory.head()

,SubCategory,count_product_view,count_purchase,product_view_to_purchase_ratios_subcategory
0,Amazon Alexa,1,0.0,0.000000
1,Arm sleeve,1,0.0,0.000000
2,Aux Cable,3,1.0,0.333333
3,Baby Shampoo,1,0.0,0.000000
4,Baby Shower gel,1,0.0,0.000000


### Dataset Modelling

##### Getting the labels 

In [121]:
# function to get the label for the row
def get_label(lst):
    
    if len(lst)==1: #it means the user only added_to_cart and haven't bought anything
        return 0
    # Convert strings to datetime objects
    cart_time = pd.to_datetime(lst[0])
    purchase_time = pd.to_datetime(lst[1])
    # Calculate time difference
    time_diff = purchase_time - cart_time
    # Define a timedelta representing 2 hours
    two_hours = pd.Timedelta(hours=2)
    # Check if the time difference is less than or equal to 2 hours
    if time_diff <= two_hours:
        return 1
    else:
        return 0



# function to get the dataset
def get_dataset(data):
    dff= data.copy()
    # filtering the data only for purchase and add_to_cart action
    data = dff[dff['Action'].isin(["purchase", "add_to_cart"])]
    data.sort_values(by=["User_id","DateTime"],inplace=True)

    # grouping rows using "User_id", "Category", "SubCategory" and creating the action list anf respective datetime list
    data_grouped = data.groupby(["User_id", "Category", "SubCategory"]).agg({
        "Action": list,
        "DateTime": lambda x: x.tolist()
    })
    data_grouped.reset_index(inplace=True)
    # filtering rows where add_to_cart action is followed by purchase action
    filtered_data = data_grouped[data_grouped['Action'].apply(lambda x: (x == ['add_to_cart', 'purchase']) or (x == ['add_to_cart'] ))]
    # getting the labels if the difference between the datetime elements in the list is less than 2 hrs
    filtered_data["Label"]=filtered_data["DateTime"].apply(get_label)

    # dropping the unwanted columns
    filtered_data.drop(columns=["Action","DateTime"],inplace=True)

    return filtered_data

In [122]:
# getting the dataset
df_base = get_dataset(df)

df_base


,User_id,Category,SubCategory,Label
0,51448162784,Camera Accessories,Camera Bag,1
1,51448162784,Camera Accessories,Camera Lens,0
2,51455512201,Men's Fashion,Shoes,0
3,51462598385,Mobile & Accessories,Aux Cable,1
4,51462598385,Mobile & Accessories,USB Cord,0
6,51463622150,Stationary,Paints,1
8,51465529647,Home Decor,Wall hangings,0
9,51468519074,Gifts,Paintings,1
12,51480378215,Electronic Appliances,Microphone,0
14,51481443687,Gardern and Outdoor,Flower plant seed,0


#### Testin ends

In [123]:
# adding no of days active feature
df_base = pd.merge(df_base,days_active,on="User_id",how="left")
df_base.head()

,User_id,Category,SubCategory,Label,days_active
0,51448162784,Camera Accessories,Camera Bag,1,3
1,51448162784,Camera Accessories,Camera Lens,0,3
2,51455512201,Men's Fashion,Shoes,0,3
3,51462598385,Mobile & Accessories,Aux Cable,1,5
4,51462598385,Mobile & Accessories,USB Cord,0,5


In [124]:
# adding RFM features for the users
users_rfm_features = df.groupby("User_id").agg({"R":"max","F":"max","M":"max","Loyalty":"max"})
df_base = pd.merge(df_base,users_rfm_features,on="User_id",how="left")
df_base.head()

,User_id,Category,SubCategory,Label,days_active,R,F,M,Loyalty
0,51448162784,Camera Accessories,Camera Bag,1,3,4.0,4.0,4.0,Platinum
1,51448162784,Camera Accessories,Camera Lens,0,3,4.0,4.0,4.0,Platinum
2,51455512201,Men's Fashion,Shoes,0,3,3.0,1.0,3.0,Gold
3,51462598385,Mobile & Accessories,Aux Cable,1,5,4.0,4.0,1.0,Platinum
4,51462598385,Mobile & Accessories,USB Cord,0,5,4.0,4.0,1.0,Platinum


In [125]:
# adding avg_time between purchase
df_base = pd.merge(df_base,avg_purchase_time[["User_id","Avg Purchase Gap"]],on="User_id",how="left")
df_base.head()

,User_id,Category,SubCategory,Label,days_active,R,F,M,Loyalty,Avg Purchase Gap
0,51448162784,Camera Accessories,Camera Bag,1,3,4.0,4.0,4.0,Platinum,184.0
1,51448162784,Camera Accessories,Camera Lens,0,3,4.0,4.0,4.0,Platinum,184.0
2,51455512201,Men's Fashion,Shoes,0,3,3.0,1.0,3.0,Gold,NaN
3,51462598385,Mobile & Accessories,Aux Cable,1,5,4.0,4.0,1.0,Platinum,56.0
4,51462598385,Mobile & Accessories,USB Cord,0,5,4.0,4.0,1.0,Platinum,56.0


In [126]:
# adding the carts_to_purchase ratios for each user
df_base = pd.merge(df_base,cart_to_purchase[["User_id","add_to_cart_to_purchase_ratios"]],on="User_id",how="left")
df_base.head()

,User_id,Category,SubCategory,Label,days_active,R,F,M,Loyalty,Avg Purchase Gap,add_to_cart_to_purchase_ratios
0,51448162784,Camera Accessories,Camera Bag,1,3,4.0,4.0,4.0,Platinum,184.0,1.0
1,51448162784,Camera Accessories,Camera Lens,0,3,4.0,4.0,4.0,Platinum,184.0,1.0
2,51455512201,Men's Fashion,Shoes,0,3,3.0,1.0,3.0,Gold,NaN,1.0
3,51462598385,Mobile & Accessories,Aux Cable,1,5,4.0,4.0,1.0,Platinum,56.0,1.0
4,51462598385,Mobile & Accessories,USB Cord,0,5,4.0,4.0,1.0,Platinum,56.0,1.0


In [127]:
# adding the wishlist_to_purchase ratios for each user
df_base = pd.merge(df_base,wishlist_to_purchase[["User_id","add_to_wishlist_to_purchase_ratios"]],on="User_id",how="left")
df_base.head()

,User_id,Category,SubCategory,Label,days_active,R,F,M,Loyalty,Avg Purchase Gap,add_to_cart_to_purchase_ratios,add_to_wishlist_to_purchase_ratios
0,51448162784,Camera Accessories,Camera Bag,1,3,4.0,4.0,4.0,Platinum,184.0,1.0,NaN
1,51448162784,Camera Accessories,Camera Lens,0,3,4.0,4.0,4.0,Platinum,184.0,1.0,NaN
2,51455512201,Men's Fashion,Shoes,0,3,3.0,1.0,3.0,Gold,NaN,1.0,NaN
3,51462598385,Mobile & Accessories,Aux Cable,1,5,4.0,4.0,1.0,Platinum,56.0,1.0,NaN
4,51462598385,Mobile & Accessories,USB Cord,0,5,4.0,4.0,1.0,Platinum,56.0,1.0,NaN


In [128]:
# adding the click_wishlist_to_purchase ratios for each user
df_base = pd.merge(df_base,wishlist_click_to_purchase[["User_id","click_wishlist_page_to_purchase_ratios"]],on="User_id",how="left")
df_base.head()

,User_id,Category,SubCategory,Label,days_active,R,F,M,Loyalty,Avg Purchase Gap,add_to_cart_to_purchase_ratios,add_to_wishlist_to_purchase_ratios,click_wishlist_page_to_purchase_ratios
0,51448162784,Camera Accessories,Camera Bag,1,3,4.0,4.0,4.0,Platinum,184.0,1.0,NaN,2.0
1,51448162784,Camera Accessories,Camera Lens,0,3,4.0,4.0,4.0,Platinum,184.0,1.0,NaN,2.0
2,51455512201,Men's Fashion,Shoes,0,3,3.0,1.0,3.0,Gold,NaN,1.0,NaN,NaN
3,51462598385,Mobile & Accessories,Aux Cable,1,5,4.0,4.0,1.0,Platinum,56.0,1.0,NaN,NaN
4,51462598385,Mobile & Accessories,USB Cord,0,5,4.0,4.0,1.0,Platinum,56.0,1.0,NaN,NaN


In [129]:
# adding the latest path for each user
df_base = pd.merge(df_base,paths,on="User_id",how="left")
df_base.head()

,User_id,Category,SubCategory,Label,days_active,R,F,M,Loyalty,Avg Purchase Gap,add_to_cart_to_purchase_ratios,add_to_wishlist_to_purchase_ratios,click_wishlist_page_to_purchase_ratios,user_path
0,51448162784,Camera Accessories,Camera Bag,1,3,4.0,4.0,4.0,Platinum,184.0,1.0,NaN,2.0,others
1,51448162784,Camera Accessories,Camera Lens,0,3,4.0,4.0,4.0,Platinum,184.0,1.0,NaN,2.0,others
2,51455512201,Men's Fashion,Shoes,0,3,3.0,1.0,3.0,Gold,NaN,1.0,NaN,NaN,search-->product_view-->read_reviews-->add_to_cart
3,51462598385,Mobile & Accessories,Aux Cable,1,5,4.0,4.0,1.0,Platinum,56.0,1.0,NaN,NaN,search-->product_view-->read_reviews-->add_to_cart
4,51462598385,Mobile & Accessories,USB Cord,0,5,4.0,4.0,1.0,Platinum,56.0,1.0,NaN,NaN,search-->product_view-->read_reviews-->add_to_cart


In [130]:
# adding category and sub-category level features carts_to_purchase ratios
df_base = pd.merge(df_base,cart_to_purchase_ratio_category[["Category","cart_to_purchase_ratios_category"]],on="Category",how="left")
df_base = pd.merge(df_base,cart_to_purchase_ratio_subcategory[["SubCategory","cart_to_purchase_ratios_subcategory"]],on="SubCategory",how="left")

In [131]:
# adding category and sub-category level features wishlist_to_purchase ratios
df_base = pd.merge(df_base,wishlist_to_purchase_ratio_category[["Category","wishlist_to_purchase_ratios_category"]],on="Category",how="left")
df_base = pd.merge(df_base,wishlist_to_purchase_ratio_subcategory[["SubCategory","wishlist_to_purchase_ratios_subcategory"]],on="SubCategory",how="left")

In [132]:
# adding category and sub-category level features click_wishlist_to_purchase ratios
df_base = pd.merge(df_base,click_wishlist_to_purchase_ratio_category[["Category","click_wishlist_to_purchase_ratios_category"]],on="Category",how="left")
df_base = pd.merge(df_base,click_wishlist_to_purchase_ratio_subcategory[["SubCategory","click_wishlist_to_purchase_ratios_subcategory"]],on="SubCategory",how="left")

In [133]:
# adding category and sub-category level features view_to_purchase ratios
df_base = pd.merge(df_base,view_to_purchase_ratio_category[["Category","product_view_to_purchase_ratios_category"]],on="Category",how="left")
df_base = pd.merge(df_base,view_to_purchase_ratio_subcategory[["SubCategory","product_view_to_purchase_ratios_subcategory"]],on="SubCategory",how="left")

In [134]:
# dropping the user_id column
df_base.drop(columns=["User_id"],inplace=True)

In [135]:
df_base.head()

,Category,SubCategory,Label,days_active,R,F,M,Loyalty,Avg Purchase Gap,add_to_cart_to_purchase_ratios,add_to_wishlist_to_purchase_ratios,click_wishlist_page_to_purchase_ratios,user_path,cart_to_purchase_ratios_category,cart_to_purchase_ratios_subcategory,wishlist_to_purchase_ratios_category,wishlist_to_purchase_ratios_subcategory,click_wishlist_to_purchase_ratios_category,click_wishlist_to_purchase_ratios_subcategory,product_view_to_purchase_ratios_category,product_view_to_purchase_ratios_subcategory
0,Camera Accessories,Camera Bag,1,3,4.0,4.0,4.0,Platinum,184.0,1.0,NaN,2.0,others,0.300000,1.0,NaN,NaN,0.30,1.0,0.600000,NaN
1,Camera Accessories,Camera Lens,0,3,4.0,4.0,4.0,Platinum,184.0,1.0,NaN,2.0,others,0.300000,1.0,NaN,NaN,0.30,NaN,0.600000,1.000000
2,Men's Fashion,Shoes,0,3,3.0,1.0,3.0,Gold,NaN,1.0,NaN,NaN,search-->product_view-->read_reviews-->add_to_cart,0.800000,1.0,8.000000,NaN,2.00,1.0,0.615385,0.500000
3,Mobile & Accessories,Aux Cable,1,5,4.0,4.0,1.0,Platinum,56.0,1.0,NaN,NaN,search-->product_view-->read_reviews-->add_to_cart,0.933333,1.0,2.333333,NaN,1.75,NaN,0.538462,0.333333
4,Mobile & Accessories,USB Cord,0,5,4.0,4.0,1.0,Platinum,56.0,1.0,NaN,NaN,search-->product_view-->read_reviews-->add_to_cart,0.933333,1.0,2.333333,NaN,1.75,NaN,0.538462,1.000000


In [139]:
# exporting the data
df_base.to_csv("data/customer_data_engineered_features.csv",index=False)